# **Project 3: О вкусной и здоровой пище**

## **Загрузка Pandas и очистка данных**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

data = pd.read_csv('data/main_task.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [2]:
# create a dict with an average Rating by city
ctr = dict()
temp = data.groupby('City').mean()[['Rating']].to_dict('index')
for city, value in temp.items():
    ctr[city] = value['Rating']
print(ctr)

{'Amsterdam': 4.1312154696132595, 'Athens': 4.2300955414012735, 'Barcelona': 3.9612289685442574, 'Berlin': 4.124593967517401, 'Bratislava': 3.9700996677740865, 'Brussels': 3.9089622641509436, 'Budapest': 4.0900735294117645, 'Copenhagen': 3.9855842185128982, 'Dublin': 4.061664190193165, 'Edinburgh': 4.0880872483221475, 'Geneva': 3.9625779625779627, 'Hamburg': 4.021074815595363, 'Helsinki': 3.9468085106382977, 'Krakow': 4.146726862302483, 'Lisbon': 4.046153846153846, 'Ljubljana': 4.112021857923497, 'London': 3.9550981413930866, 'Luxembourg': 3.914285714285714, 'Lyon': 3.9484304932735426, 'Madrid': 3.800836550836551, 'Milan': 3.8248945147679323, 'Munich': 4.047032474804031, 'Oporto': 4.178362573099415, 'Oslo': 3.8935064935064934, 'Paris': 3.9424137226873595, 'Prague': 4.032571032571033, 'Rome': 4.230269489894129, 'Stockholm': 3.870121951219512, 'Vienna': 4.078044596912521, 'Warsaw': 4.089408528198074, 'Zurich': 4.039962825278811}


In [3]:
def calculate_mean_score(row):
    return ctr.get(row['City'])

In [4]:
def replace_price_range(row):
    if row['Price Range'] == '$':
        return 10
    elif row['Price Range'] == '$$ - $$$':
        return 20
    else:
        return 30

In [5]:
allCuisines = []

def get_cuisines(cuisines):
    uniqueCuisines = list()
    for cuisine in cuisines.split(', '): 
        uniqueCuisines.append(cuisine)
    if len(uniqueCuisines) == 0:
        uniqueCuisines.append('Regional Cuisine')
    return uniqueCuisines
        
def cuisine_styles_count(row):
    global allCusines
    cuisines = get_cuisines(row['Cuisine Style'])
    
    if row['Cuisine Style'] != 'NaN':    
        cuisines = get_cuisines(row['Cuisine Style'])
        allCuisines.extend(cuisines)
        cuisines_count = len(cuisines)
    else:
        cuisines_count = 1

    return cuisines_count

In [6]:
# add a default cuisine
#cuisine_values = {'Cuisine Style': '[\'Vegetarian Friendly\']'}
cuisine_values = {'Cuisine Style': '[\'Regional Cuisine\']'} 
data = data.fillna(value=cuisine_values)

# replace bad json values
data['Cuisine Style'] = data['Cuisine Style'].str.replace('[', '').str.replace(']', '').str.replace('\'', '')

# find style and count for each cousine
data['Cuisine Style NAN'] = data['Cuisine Style'].isna()
data['Cuisine Style'] = data['Cuisine Style'].fillna('NaN')
data['Cuisines Count'] = data.apply(cuisine_styles_count, axis=1)

most_popular_cousine = pd.Series(allCuisines).value_counts().index[0]
average_cousines_count = np.round(data['Cuisines Count'].mean())

print("Most popular cousine - " + str(most_popular_cousine))
print("Average cousines count - " + str(average_cousines_count))

# flags for popular cousine
data['isMostPopCuisine'] = data['Cuisine Style'].apply(lambda x: 1 if most_popular_cousine in x else 0 )
data['isMultyCuisine'] = data['Cuisines Count'].apply(lambda x: 1 if  x >= average_cousines_count else 0 )

# get dummies for cuisines
data['Cuisine Style'] = data['Cuisine Style'].str.replace('[', '').str.replace(']', '').str.replace('\'', '')
dummies = data['Cuisine Style'].str.get_dummies(sep=', ')

data = data.join(dummies)

# add a default price range instead of null
price_values = {'Price Range': '$$ - $$$'}
data = data.fillna(value=price_values)
data['Prices'] = data.apply(replace_price_range, axis=1)

# add a new column - mean rating per city 
data['CtrCity'] = data.apply(calculate_mean_score, axis=1)

# create dummies from City column
data = pd.get_dummies(data, columns=['City'], dummy_na=True)

# add a mean value to numeric column instead of null values
review_mean = data['Number of Reviews'].mean(skipna = True) 

review_values = {'Number of Reviews': review_mean}
data = data.fillna(value=review_values)

Most popular cousine - Vegetarian Friendly
Average cousines count - 3.0


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [7]:
# drop non numeric columns
data = data.drop(['Price Range', 'Cuisine Style', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = data.drop(['Restaurant_id', 'Rating'], axis = 1)
y = data['Rating']

In [8]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [9]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [10]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [11]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [12]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
# Hint: The less, the better 
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
# 0.42473367440476195 - default value
# 0.224704  - the best with mean value per city
# 0.2084965 - with dummies

MAE: 0.21212149999999996
